## Against / Creating New Validators

You will have seen use the `against` module as part of [selection validation](../basics/selection-validate.html) and [column validation](./validation.html).

The provided `against` module is a convenience wrapper for accessing a datachef `Valdiator`.
    
A `Validator` in datachef is any class that inherits from [datachef.against.implementation.base.BaseValidator](https://github.com/mikeAdamss/datachef/blob/main/datachef/against/implementations/base.py).

A a convenience the code for for this class is shown below (don't worry, it's actually very straight forward).

![](./images/against-base.png)

You can also [view this class on github](https://github.com/mikeAdamss/datachef/blob/main/datachef/against/implementations/base.py).